In [1]:
from pandas import DataFrame
from IPython.display import HTML
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import pandas as pd 
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

pd.options.display.max_rows = 4000

frequent_spamwords = ['trump', 'news', 'polic', 'break', 'man', 'obama', 'polit', 'world', 'kill',
'sport','sport','state', 'amp', 'hillari','fire', 'local','against',
'white', 'support', 'report', 'u.s.','shoot','attack','hous','american',
'woman','offic','america','media','arrest','cop','liber','home','democrat',
'down','crash','elect','shot','gop','dead','court','suspect','investig',
'crime','busi','death','maga','fight','gun','north','south','million',
'tcot','women','realdonaldtrump','nation','doe','bill','counti'] 

frequent_qualitywords = ['love', 'thank','happi', 'good','got','some','think','feel','come',
'pleas','pleas','u','birthday','na','realli','life','been', 'even','ca',
'girl','great','last','much',"'ll","'ve",'never','best','lol','still',
'alway','way','give','post','too','check','start','am','someon','im',
'2016','where','job','off','week','miss','shit','lot','talk','follow',
'free','guy','ani','>','music','stop','fuck','becaus','leav','person',
'photo']

def preprocess(filename):
    df = pd.read_csv(filename, encoding= 'unicode_escape')
    df.fillna(0)
    #convert numbers
    df['Type'] = df['Type'].str.replace('Spam','1')
    df['Type'] = df['Type'].str.replace('Quality','0')
    #NaN -> 0
    df['Type'] = df['Type'].fillna(0)
    df['following'] = df['following'].fillna(0)
    df['actions'] = df['actions'].fillna(0)
    df['is_retweet'] = df['is_retweet'].fillna(0)
    df['followers'] = df['followers'].fillna(0)
    df['Reputation'] = df['Reputation'].str.replace('#DIV/0!','0')
    
    
    Tweets = df["Tweet"]
   
    #Count spam and quality words in tweets
    #Count hashtags and links in tweets
    words = []
    wordrateCol = []
    hashtagsCol = []
    linksCol = []
    ps = PorterStemmer()
    for tweet in Tweets:
        wordrate = 0
        hashtags = 0
        links = 0
        for word in word_tokenize(tweet):
            stemmed_word = ps.stem(word)
            if stemmed_word in frequent_spamwords:
                wordrate = wordrate + 1
            elif stemmed_word in frequent_qualitywords:
                wordrate = wordrate - 1
            if stemmed_word == '#':
                hashtags = hashtags + 1
            if stemmed_word == 'http':
                links = links + 1

        wordrateCol.append(wordrate)
        hashtagsCol.append(hashtags)
        linksCol.append(links)

    df['wordrate'] = wordrateCol
    df['hashtags'] = hashtagsCol
    df['links'] = linksCol
    

    #extract feature vector and result vector
    df = df.drop('Tweet',axis='columns')
    return df

def chooseFeatures(df,feature_list=["wordrate",'hashtags','links','following','followers','Reputation','actions','is_retweet']):
    target = df.Type
    inputs = df.drop('Type',axis='columns')
    inputs = inputs.drop('Id',axis='columns')
    #X = inputs
    X = df[df.columns[df.columns.isin(feature_list)]]
    #print(X)
    y = target
    
    return X,y
    

def trainModel(X,y,m):
    score = []
    for i in range(1):
        X_train, X_test, y_train, y_test = train_test_split(X,y, stratify=y,test_size=0.1)
        if m == 1:
            model = RandomForestClassifier(n_estimators=30)
        elif m == 2:  
            model = GaussianNB()
        model.fit(X_train, y_train)
        score.append(model.score(X_test, y_test))
    print("Score = ")
    print(sum(score)/len(score))
    


In [3]:
df = preprocess("P2Sample2.csv")
X,y = chooseFeatures(df)
trainModel(X,y,2)

Score = 
0.581453634085213


<ipykernel.eventloops.loop_wx.<locals>.IPWxApp object at 0x0000018B3DA778B0>


In [231]:
#df[:20]

# ht = HTML(df.to_html())
# display(ht)
#print(df["Type"].value_counts())

,Id,following,followers,Reputation,actions,is_retweet,Type,wordrate,hashtags,links
0,3697,3319.0,611.0,0.155470738,294.0,0.0,1,1,1,2
1,10740,4840.0,1724.0,0.262644729,1522.0,0.0,1,0,0,1
2,9572,4435.0,16041.0,0.783404962,27750.0,0.0,1,0,0,1
3,10963,9025.0,20165.0,0.690818774,6331.0,0.0,1,1,0,2
4,5401,780.0,897.0,0.534883721,4792.0,1.0,1,-1,2,2
5,7636,1893.0,1651.0,0.465857788,3564.0,1.0,1,3,3,2
6,6908,7981.0,12815.0,0.616224274,13601.0,1.0,1,0,0,1
7,10433,85.0,73.0,0.462025317,434.0,0.0,1,0,3,0
8,4941,856.0,1093.0,0.560800411,6219.0,1.0,1,0,0,2
9,7237,456.0,821.0,0.642913078,5797.0,1.0,1,0,1,0


In [ ]:

# spam = df.loc[df["Type"]=='1']
# quality = df.loc[df["Type"]=='0']

# spamTweets = spam["Tweet"]
# spamwords = []
# for sentence in spamTweets:
#     for word in word_tokenize(sentence):
#         stemmed_word = ps.stem(word)
#         spamwords.append(stemmed_word)
# spamwords

frequent_spamwords = ['trump', 'news', 'polic', 'break', 'man', 'obama', 'polit', 'world', 'kill',
'sport','sport','state', 'amp', 'hillari','fire', 'local','against',
'white', 'support', 'report', 'u.s.','shoot','attack','hous','american',
'woman','offic','america','media','arrest','cop','liber','home','democrat',
'down','crash','elect','shot','gop','dead','court','suspect','investig',
'crime','busi','death','maga','fight','gun','north','south','million',
'tcot','women','realdonaldtrump','nation','doe','bill','counti'] 

# qualityTweets = quality["Tweet"]
# qualitywords = []
# for sentence in qualityTweets:
#     for word in word_tokenize(sentence):
#         stemmed_word = ps.stem(word)
#         qualitywords.append(stemmed_word)
# qualitywords

frequent_qualitywords = ['love', 'thank','happi', 'good','got','some','think','feel','come',
'pleas','pleas','u','birthday','na','realli','life','been', 'even','ca',
'girl','great','last','much',"'ll","'ve",'never','best','lol','still',
'alway','way','give','post','too','check','start','am','someon','im',
'2016','where','job','off','week','miss','shit','lot','talk','follow',
'free','guy','ani','>','music','stop','fuck','becaus','leav','person',
'photo']

# cq = Counter(qualitywords)
# print(cq.most_common(200))
# print("__________________________________________________-")
# c = Counter(spamwords)
# print(c.most_common(200))


In [ ]:
# Tweets = df["Tweet"]

In [ ]:
# #Count spam and quality words in tweets
# #Count hashtags and links in tweets
# words = []
# wordrateCol = []
# hashtagsCol = []
# linksCol = []
# ps = PorterStemmer()
# for tweet in Tweets:
#     wordrate = 0
#     hashtags = 0
#     links = 0
#     for word in word_tokenize(tweet):
#         stemmed_word = ps.stem(word)
#         if stemmed_word in frequent_spamwords:
#             wordrate = wordrate + 1
#         elif stemmed_word in frequent_qualitywords:
#             wordrate = wordrate - 1
#         if stemmed_word == '#':
#             hashtags = hashtags + 1
#         if stemmed_word == 'http':
#             links = links + 1
    
#     wordrateCol.append(wordrate)
#     hashtagsCol.append(hashtags)
#     linksCol.append(links)

# df['wordrate'] = wordrateCol
# df['hashtags'] = hashtagsCol
# df['links'] = linksCol

# ht = HTML(df[:50].to_html())
# display(ht)






In [ ]:
# df = df.drop('Tweet',axis='columns')
# target = df.Type
# inputs = df.drop('Type',axis='columns')
# inputs = inputs.drop('Id',axis='columns')


In [ ]:
# ht = HTML(df[:50].to_html())
# display(ht)

In [ ]:
#inputs.columns[inputs.isna().any()]

In [ ]:
# X = inputs
# # X = df[df.columns[df.columns.isin(["wordrate",'hashtags','links'])]]
# # print(X)
# y = target

In [ ]:
from sklearn.model_selection import train_test_split

score = []
for i in range(1):
    X_train, X_test, y_train, y_test = train_test_split(X,y, stratify=y,test_size=0.1)
    from sklearn.ensemble import RandomForestClassifier
    model = RandomForestClassifier(n_estimators=30)
    model.fit(X_train, y_train)
    score.append(model.score(X_test, y_test))
print("Score = ")
print(sum(score)/len(score))


In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
y_predicted = model.predict(X_test)
HTML(X_test.to_html())
print(X_test[:10])
print(y_predicted[:10])

In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test, y_predicted)
matrix

In [ ]:
len(y_test)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sn
plt.figure(figsize=(10,7))
sn.heatmap(matrix, annot=True)
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
y_test[:15]

In [ ]:
y_predicted[:15]